In [5]:
import keras
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from matplotlib import pyplot as plt
import os

from collections import defaultdict
from PIL import Image

np.random.seed(123) # for testing

image_folder_path = r'C:\Users\azada\Documents\Uni Files\Non-Academic Uni Files\Dr. Imran Research Work\Data Sets\Spoken Digit Example\Spectros'


training_specs = []
testing_specs = []

training_labels = []
testing_labels = []


file_paths = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f)) and '.png' in f]

for file_name in file_paths:
    spec_index = file_name.rsplit("_", 1)[1].rsplit(".", 1)[0]
    label = file_name[0]
    spectrogram = Image.open(image_folder_path + '/' + file_name)
    spectrogram = spectrogram.convert('RGB')
    spectrogram = spectrogram.resize((224,224))
    spectrogram = np.array(spectrogram)
    if int(spec_index) <= 4:
        testing_specs.append(spectrogram)
        testing_labels.append(label)
    else:
        training_specs.append(spectrogram)
        training_labels.append(label)

training_specs = np.array(training_specs)
testing_specs = np.array(testing_specs)
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)


training_specs = training_specs.astype('float32') # convert types to float
testing_specs = testing_specs.astype('float32')
training_specs /= 255 # normalize data to a 0-1 value
testing_specs /= 255


training_labels = np_utils.to_categorical(training_labels, 10) # convert data to categories
testing_labels = np_utils.to_categorical(testing_labels, 10)



In [10]:

base_model = keras.applications.mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3))

for layer in base_model.layers: # here we freeze the layers from mobilenet
    layer.trainable=False

# here we add our own final layers
x=base_model.output 
x=Flatten()(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(4096,activation='relu')(x) 
preds=Dense(10,activation='softmax')(x) #final layer with softmax activation. Here '10' denotes number of classes

model=Model(inputs=base_model.input,outputs=preds)


    



In [11]:


#Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Train the model
model.fit(training_specs, training_labels, 
          batch_size=8, nb_epoch=2, verbose=1)

#Test the results
score = model.evaluate(testing_specs, testing_labels, verbose=0)

print(score)


Instructions for updating:
Use tf.cast instead.


C:\Users\azada\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/2
1800/1800 [==============================] - 322s 179ms/step - loss: 14.3292 - accuracy: 0.1028
Epoch 2/2
1800/1800 [==============================] - 342s 190ms/step - loss: 14.4974 - accuracy: 0.1006
[14.506285858154296, 0.10000000149011612]
